# HW1. Проектирование API

### Автор: *Кузнецов Кирилл Игоревич*
### Группа: М08-401НД
### Дата: 16.10.2025

Это задание выполняется в рамках модуля 1 «Проектирование API». Вы закрепите навыки разработки API, используя подход сode-first, затем будете придерживаться подхода API-first

> Чтобы получить максимальный балл, убедитесь, что ваш ноутбук запускается с нуля, структура понятна, а в выводах вы объясняете свои решения.  

---
## Подготовка окружения


1. Работа производилась на локальной машине

2. В процессе реализации была создана виртуальная среда
`conda create --name devops_env  python=3.11 `

In [ ]:
# Для установки в zsh и обхода спецссимволов немного модернизируем команду
# !pip install 'fastapi[all]' 'uvicorn[standard]'

Теперь, когда у нас установлены необходимые библиотеки, мы можем приступить к созданию нашего первого приложения FastAPI.

---
# Задание 1

Вам необходимо самостоятельно создать веб-сервер на основе FastAPI.

Задача: создать файл `main.py`, который будет содержать наш код API с четырьмя методами HTTP.



In [53]:
%%writefile main.py

import os
import random
from typing import Union
from fastapi import FastAPI, HTTPException, Request, Response, status, Header, Cookie
from fastapi.responses import JSONResponse
from pydantic import BaseModel
import uvicorn
import threading
import requests
import time
import yaml
import subprocess
import wget


# 1. Создаем объект класса для инициалицации сервера
app = FastAPI()

# 2. Имитация бд через словарь
db = dict()

# 3. Дата-классы
class Item(BaseModel):
    name: str
    description: str = None
    price: float

class ItemConfidential(BaseModel):
    name: str
    description: str = None


# 4. Прокинем методы в ассинхронном режиме
# все текстовки на английском изза utf-8

# Нулевой эндпоинт
@app.get("/")
async def read_root():
    return {"message":"Everything seems OK!! Server is running",
            "/json_data":"Endpoint with JSON responce",
            "/error":"Randomly generated error code"}

# GET /items - вывод всех айтемов
@app.get("/items", response_model=list[Item])
async def list_items():
    return list(db.values())

# POST /items - cоздание единичного айтема
@app.post("/items", status_code=status.HTTP_201_CREATED, response_model=ItemConfidential)
async def create_item(item: Item):
    item_id = len(db) + 1
    db[item_id] = item
    return item 

# GET /items/{item_id} - тащим айтем по айдишнику
@app.get("/items/{item_id}", response_model=Item)
async def get_item(item_id: int, q: Union[str, None] = None):
    if item_id not in db:
        raise HTTPException(status_code=404, detail="Item not found")
    return db[item_id]

# PUT /items/{item_id} - обновляем айтем 
@app.put("/items/{item_id}", response_model=Item)
async def update_item_put(item_id: int, item: Item):
    if item_id not in db:
        raise HTTPException(status_code=404, detail="Item not found")
    db[item_id] = item
    return item

# PATCH /items/{item_id} - обновляем айтем 
@app.patch("/items/{item_id}", response_model=Item)
async def update_item(item_id: int, item: Item):
    if item_id not in db:
        raise HTTPException(status_code=404, detail="Item not found")
    db[item_id] = item
    return item

# DELETE /items/{item_id} - удаляем айтем
@app.delete("/items/{item_id}", status_code=status.HTTP_204_NO_CONTENT)
async def delete_item(item_id: int):
    if item_id not in db:
        raise HTTPException(status_code=404, detail="Item not found")
    del db[item_id]
    return Response(status_code=status.HTTP_204_NO_CONTENT)


Writing main.py


1. Фиксируем режим ячейки в запись main.py файла как того требует документация FastApi
2. Создаем экземпляр класса FastApi и бд хранащуюся в оперативной памяти
3. Создаем датаклассы для прокидывания типизации в методы
4. Создаем основные HTTP методы через декорирование ассинхронных функций встраивая обработку ошибок

---
# Задание 2

Сервер должен отвечать валидным JSON на эндпоинте /json_data.

Задача: создать новый эндпоинт /json_data и подключить компонент JSONResponse на этом эндпоинте. Содержание JSONa не важно, главное, чтобы он был валидным.


In [54]:
%%writefile -a main.py

# Эндпоинт ответа json файлом
@app.get("/json_data", response_class=JSONResponse)
async def get_json_data():
    return {"message": "This is valid JSON", "data": [1, 2, 3], "success": True}

Appending to main.py


1. Режим ячейка на запись в конец mainpy
2. Мы добавили эндпоинт /json_data, возвращающий валидный JSON через JSONResponse

---
# Задание 3

Обработка ошибок с использованием простого HTTPException.

Задачи:
1. Изучите стандартные коды ошибок.
2. Выберите любое случайное число в диапазоне 400—526.

> Чтобы возвращать ошибки с соответствующими HTTP-статусами вам нужно:
> 1. подключить класс `HTTPException` и просто выдать любой [код ошибки](https://ru.wikipedia.org/wiki/%D0%A1%D0%BF%D0%B8%D1%81%D0%BE%D0%BA_%D0%BA%D0%BE%D0%B4%D0%BE%D0%B2_%D1%81%D0%BE%D1%81%D1%82%D0%BE%D1%8F%D0%BD%D0%B8%D1%8F_HTTP) (от 400 до 526);
> 2. создать маршрут /error, который будет генерировать ошибку с этим кодом в ответ на любой запрос.

В комментариях напишите, как вы понимаете, что означает выбранный вами код ошибки.

In [55]:
%%writefile -a main.py 


# Генерируем рандомный код ошибки сервера
# Описание: Рандомно сгенерированный код: status_code
# так как я решил не фиксировать сид а брать из всего диапазона,
# то формируем обобзенный ответ:
# все что выше 500-сервер, все что ниже клиент
status_code = random.randint(400, 526)

if 500 <= status_code <= 599:
    error_detail = f"Internal server proplem ({status_code})."
else:
    error_detail = f"Client side problem {status_code} - check your query."

# Эндпоинт /error, возвращающий HTTP-ошибку
@app.get("/error")
async def trigger_error():
    raise HTTPException(status_code=401, detail=error_detail)

Appending to main.py


С помощью HTTPException реализовали возврат HTTP-ошибок в диапазоне 400-526, создав автоматическую заглушку для целого спектра кодов

---
# Задание 4

Создание автодокументации для API: FastAPI автоматически генерирует документацию API в формате OpenAPI и предоставляет интерфейс Swagger UI для ее просмотра

*Поскольку код коллаба выполняется внутри виртуальной машины, у которой нет внешнего IP-адреса, вам потребуется создать тоннель, чтобы получить внешний IP-адрес.*

*Зарегистрируйтесь в личном кабинете https://xtunnel.ru/ и скопируйте бесплатную лицензию (секретный ключ API)*

Если возникают сложности, используйте локальную версию коллаба.
```bash
pip install notebook
jupyter notebook
```



>Так как я решил делать на локальной машине а не в колабе то пришлось немного поизголяться.

>В частности в выбранном мной подходе (из юпитера запускаем только испольняемый файл), мне пришлось отходить от использования скриптов через ! и запихивать их в строки через subprocess, плюс добавил выбор бинарников по платформе


In [57]:
%%writefile -a main.py 

if __name__ == "__main__":
    # Берем api_key из конфигурационного файла
    with open("config.yaml", "r", encoding="utf-8") as f:
        XTUNNEL_API_KEY = yaml.safe_load(f)["KEY"]
    
    # Создадим папку для xtunnel
    os.makedirs("./xtunnel", exist_ok=True)
        
    # Для запуска из консоли
    # platform = input("Choose your platform (osx, linux, win), note linux/win on x64 not ARM")
    
    # Для запуска из юпитера так как нет доступа к вводу
    platform = "osx"
    
    base_url = "https://github.com/xtunnel-dev/xtunnel-binaries/raw/refs/heads/main/1.0.20/"
    
    if platform == "osx":
        binary = base_url + "xtunnel.osx-arm64.1.0.20.zip"
    elif platform == "linux":
        binary = base_url + "xtunnel.linux-x64.1.0.20.zip"
    elif platform == "win":
        binary = base_url + "xtunnel.win-x64.1.0.20.zip"
    else:
        raise ValueError("Unsupported platform")
        
    # Скачиваем через subprocess(так как запускаем из py-файла и bash скрипты дадут ошибку)
    subprocess.run([
        "curl", "-L", "-o", "./xtunnel/xt.zip",
        binary
    ])
    
    # Распаковываем
    subprocess.run(["unzip", "-o", "./xtunnel/xt.zip", "-d", "./xtunnel/"])
    
    # Даём права на выполнение (macOS/Linux)
    os.chmod("./xtunnel/xtunnel", 0o755)
    
    # Регистрируем
    subprocess.run(["./xtunnel/xtunnel", "register", XTUNNEL_API_KEY])
    xtunnel_proc = subprocess.Popen(["./xtunnel/xtunnel", "http", "8090"])
    
    # Запускаем сервер и туннель
    uvicorn_proc = subprocess.Popen(["uvicorn", "main:app", "--port", "8090", "--host", "0.0.0.0"])
    time.sleep(5)
    xtunnel_proc = subprocess.Popen(["./xtunnel/xtunnel", "http", "8090"])

    # Ждём завершения
    try:
        xtunnel_proc.wait()
    except KeyboardInterrupt:
        uvicorn_proc.terminate()
        xtunnel_proc.terminate()

Appending to main.py


In [58]:
# Запускаем сервер
!python main.py

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 33.3M  100 33.3M    0     0  12.0M      0  0:00:02  0:00:02 --:--:-- 22.6M
Archive:  ./xtunnel/xt.zip
  inflating: ./xtunnel//xtunnel-cert.cer  
  inflating: ./xtunnel//xtunnel      
=Registration completed successfully
INFO:     Started server process [97126]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8090 (Press CTRL+C to quit)
=Status: Connecting
Public address: 
Target address: http://localhost:8090


Press Ctrl+C to exit
=Status: Connecting
Public address: 
Target address: http://localhost:8090


Press Ctrl+C to exit
Status: Connecting
Public address: https://7d1cd78b-4c0a-4241-ab4b-fae969d8b501.tunnel4.com
Public address: http://7d1cd78b-4c0a-4241-ab4b-fae96

Добавляем в конец main.py запуск сервера и запускаем его из ноутбука


## Задача. Чтобы проверить документацию, выполните следующие шаги:
1. Запустите сервер FastAPI с помощью команды ниже.
2. Откройте браузер и перейдите по адресу `http://127.0.0.1:8000/docs` для просмотра документации в Swagger UI.
3. Для просмотра документации в формате OpenAPI перейдите по адресу `http://127.0.0.1:8000/openapi.json`

In [ ]:
# Пришлось использовать json форматтер что бы было удобочитаемо https://bi-data.ru/tools/json/
'''
{
  "openapi": "3.1.0",
  "info": {
    "title": "FastAPI",
    "version": "0.1.0"
  },
  "paths": {
    "/": {
      "get": {
        "summary": "Read Root",
        "operationId": "read_root__get",
        "responses": {
          "200": {
            "description": "Successful Response",
            "content": {
              "application/json": {
                "schema": {}
              }
            }
          }
        }
      }
    },
    "/items": {
      "get": {
        "summary": "List Items",
        "operationId": "list_items_items_get",
        "responses": {
          "200": {
            "description": "Successful Response",
            "content": {
              "application/json": {
                "schema": {
                  "items": {
                    "$ref": "#/components/schemas/Item"
                  },
                  "type": "array",
                  "title": "Response List Items Items Get"
                }
              }
            }
          }
        }
      },
      "post": {
        "summary": "Create Item",
        "operationId": "create_item_items_post",
        "requestBody": {
          "content": {
            "application/json": {
              "schema": {
                "$ref": "#/components/schemas/Item"
              }
            }
          },
          "required": true
        },
        "responses": {
          "201": {
            "description": "Successful Response",
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/ItemConfidential"
                }
              }
            }
          },
          "422": {
            "description": "Validation Error",
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/HTTPValidationError"
                }
              }
            }
          }
        }
      }
    },
    "/items/{item_id}": {
      "get": {
        "summary": "Get Item",
        "operationId": "get_item_items__item_id__get",
        "parameters": [
          {
            "name": "item_id",
            "in": "path",
            "required": true,
            "schema": {
              "type": "integer",
              "title": "Item Id"
            }
          },
          {
            "name": "q",
            "in": "query",
            "required": false,
            "schema": {
              "anyOf": [
                {
                  "type": "string"
                },
                {
                  "type": "null"
                }
              ],
              "title": "Q"
            }
          }
        ],
        "responses": {
          "200": {
            "description": "Successful Response",
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Item"
                }
              }
            }
          },
          "422": {
            "description": "Validation Error",
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/HTTPValidationError"
                }
              }
            }
          }
        }
      },
      "put": {
        "summary": "Update Item Put",
        "operationId": "update_item_put_items__item_id__put",
        "parameters": [
          {
            "name": "item_id",
            "in": "path",
            "required": true,
            "schema": {
              "type": "integer",
              "title": "Item Id"
            }
          }
        ],
        "requestBody": {
          "required": true,
          "content": {
            "application/json": {
              "schema": {
                "$ref": "#/components/schemas/Item"
              }
            }
          }
        },
        "responses": {
          "200": {
            "description": "Successful Response",
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Item"
                }
              }
            }
          },
          "422": {
            "description": "Validation Error",
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/HTTPValidationError"
                }
              }
            }
          }
        }
      },
      "patch": {
        "summary": "Update Item",
        "operationId": "update_item_items__item_id__patch",
        "parameters": [
          {
            "name": "item_id",
            "in": "path",
            "required": true,
            "schema": {
              "type": "integer",
              "title": "Item Id"
            }
          }
        ],
        "requestBody": {
          "required": true,
          "content": {
            "application/json": {
              "schema": {
                "$ref": "#/components/schemas/Item"
              }
            }
          }
        },
        "responses": {
          "200": {
            "description": "Successful Response",
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Item"
                }
              }
            }
          },
          "422": {
            "description": "Validation Error",
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/HTTPValidationError"
                }
              }
            }
          }
        }
      },
      "delete": {
        "summary": "Delete Item",
        "operationId": "delete_item_items__item_id__delete",
        "parameters": [
          {
            "name": "item_id",
            "in": "path",
            "required": true,
            "schema": {
              "type": "integer",
              "title": "Item Id"
            }
          }
        ],
        "responses": {
          "204": {
            "description": "Successful Response"
          },
          "422": {
            "description": "Validation Error",
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/HTTPValidationError"
                }
              }
            }
          }
        }
      }
    },
    "/json_data": {
      "get": {
        "summary": "Get Json Data",
        "operationId": "get_json_data_json_data_get",
        "responses": {
          "200": {
            "description": "Successful Response",
            "content": {
              "application/json": {
                "schema": {}
              }
            }
          }
        }
      }
    },
    "/error": {
      "get": {
        "summary": "Trigger Error",
        "operationId": "trigger_error_error_get",
        "responses": {
          "200": {
            "description": "Successful Response",
            "content": {
              "application/json": {
                "schema": {}
              }
            }
          }
        }
      }
    }
  },
  "components": {
    "schemas": {
      "HTTPValidationError": {
        "properties": {
          "detail": {
            "items": {
              "$ref": "#/components/schemas/ValidationError"
            },
            "type": "array",
            "title": "Detail"
          }
        },
        "type": "object",
        "title": "HTTPValidationError"
      },
      "Item": {
        "properties": {
          "name": {
            "type": "string",
            "title": "Name"
          },
          "description": {
            "type": "string",
            "title": "Description"
          },
          "price": {
            "type": "number",
            "title": "Price"
          }
        },
        "type": "object",
        "required": [
          "name",
          "price"
        ],
        "title": "Item"
      },
      "ItemConfidential": {
        "properties": {
          "name": {
            "type": "string",
            "title": "Name"
          },
          "description": {
            "type": "string",
            "title": "Description"
          }
        },
        "type": "object",
        "required": [
          "name"
        ],
        "title": "ItemConfidential"
      },
      "ValidationError": {
        "properties": {
          "loc": {
            "items": {
              "anyOf": [
                {
                  "type": "string"
                },
                {
                  "type": "integer"
                }
              ]
            },
            "type": "array",
            "title": "Location"
          },
          "msg": {
            "type": "string",
            "title": "Message"
          },
          "type": {
            "type": "string",
            "title": "Error Type"
          }
        },
        "type": "object",
        "required": [
          "loc",
          "msg",
          "type"
        ],
        "title": "ValidationError"
      }
    }
  }
}
'''

---
# Задание 5

Автодокументаци API хороша для маленьких проектов. Сейчас вам нужно полученый в задании 4 JSON вставить в редактор [Swagger](https://editor.swagger.io/), добавить проектируемый маршрут с ответом в формате XML и сохранить описание в YAML.

Задачи:
1. Откройте редактор Swagger, вставьте JSON (ответьте ОК на запрос Would you like to convert your JSON into YAML?)
2. Добавьте 1 эндпоинт.
3. Скопируйте получившееся описание в YAML и вставьте в ячейку ниже.

---
При копировании полученного json файла в swagger были получены следующие ошибки:


>Structural error at openapi should match pattern "^3\.0\.\d(-.+)?$" pattern: ^3\.0\.\d(-.+)?$ Jump to line 1

> Structural error at paths./items/{item_id}.get.parameters.1.schema.anyOf.1.type
should be equal to one of the allowed values allowedValues: array, boolean, integer, number, object, string
Jump to line 69

Первая ошибка возникает из-за того что Swager ожидает версию ниже (например 3.0.0)

Вторая возникает из первой, поиск обхода ошибки для формирования yaml файла привел к решению:
- *type: 'null'* заменить на *nullable: true*

---


In [ ]:
# Итоговый отредактированный yaml из swagger + допэндпоинт добавленный на этом этапе:
# /xml_data - на основе уже существующего /json_data, применив 

"""
openapi: 3.0.3
info:
  title: FastAPI
  version: 0.1.0
paths:
  /:
    get:
      summary: Read Root
      operationId: read_root__get
      responses:
        '200':
          description: Successful Response
          content:
            application/json:
              schema: {}
  /items:
    get:
      summary: List Items
      operationId: list_items_items_get
      responses:
        '200':
          description: Successful Response
          content:
            application/json:
              schema:
                items:
                  $ref: '#/components/schemas/Item'
                type: array
                title: Response List Items Items Get
    post:
      summary: Create Item
      operationId: create_item_items_post
      requestBody:
        content:
          application/json:
            schema:
              $ref: '#/components/schemas/Item'
        required: true
      responses:
        '201':
          description: Successful Response
          content:
            application/json:
              schema:
                $ref: '#/components/schemas/ItemConfidential'
        '422':
          description: Validation Error
          content:
            application/json:
              schema:
                $ref: '#/components/schemas/HTTPValidationError'
  /items/{item_id}:
    get:
      summary: Get Item
      operationId: get_item_items__item_id__get
      parameters:
        - name: item_id
          in: path
          required: true
          schema:
            type: integer
            title: Item Id
        - name: q
          in: query
          required: false
          schema:
            anyOf:
              - type: string
              - nullable: true
            title: Q
      responses:
        '200':
          description: Successful Response
          content:
            application/json:
              schema:
                $ref: '#/components/schemas/Item'
        '422':
          description: Validation Error
          content:
            application/json:
              schema:
                $ref: '#/components/schemas/HTTPValidationError'
    put:
      summary: Update Item Put
      operationId: update_item_put_items__item_id__put
      parameters:
        - name: item_id
          in: path
          required: true
          schema:
            type: integer
            title: Item Id
      requestBody:
        required: true
        content:
          application/json:
            schema:
              $ref: '#/components/schemas/Item'
      responses:
        '200':
          description: Successful Response
          content:
            application/json:
              schema:
                $ref: '#/components/schemas/Item'
        '422':
          description: Validation Error
          content:
            application/json:
              schema:
                $ref: '#/components/schemas/HTTPValidationError'
    patch:
      summary: Update Item
      operationId: update_item_items__item_id__patch
      parameters:
        - name: item_id
          in: path
          required: true
          schema:
            type: integer
            title: Item Id
      requestBody:
        required: true
        content:
          application/json:
            schema:
              $ref: '#/components/schemas/Item'
      responses:
        '200':
          description: Successful Response
          content:
            application/json:
              schema:
                $ref: '#/components/schemas/Item'
        '422':
          description: Validation Error
          content:
            application/json:
              schema:
                $ref: '#/components/schemas/HTTPValidationError'
    delete:
      summary: Delete Item
      operationId: delete_item_items__item_id__delete
      parameters:
        - name: item_id
          in: path
          required: true
          schema:
            type: integer
            title: Item Id
      responses:
        '204':
          description: Successful Response
        '422':
          description: Validation Error
          content:
            application/json:
              schema:
                $ref: '#/components/schemas/HTTPValidationError'
  /json_data:
    get:
      summary: Get Json Data
      operationId: get_json_data_json_data_get
      responses:
        '200':
          description: Successful Response
          content:
            application/json:
              schema: {}
  /xml_data:
    get:
      summary: Get XML Data
      operationId: get_xml_data_xml_data_get
      responses:
        '200':
          description: Successful Response with XML content
          content:
            application/xml:
              schema:
                type: string
  /error:
    get:
      summary: Trigger Error
      operationId: trigger_error_error_get
      responses:
        '200':
          description: Successful Response
          content:
            application/json:
              schema: {}
components:
  schemas:
    HTTPValidationError:
      properties:
        detail:
          items:
            $ref: '#/components/schemas/ValidationError'
          type: array
          title: Detail
      type: object
      title: HTTPValidationError
    Item:
      properties:
        name:
          type: string
          title: Name
        description:
          type: string
          title: Description
        price:
          type: number
          title: Price
      type: object
      required:
        - name
        - price
      title: Item
    ItemConfidential:
      properties:
        name:
          type: string
          title: Name
        description:
          type: string
          title: Description
      type: object
      required:
        - name
      title: ItemConfidential
    ValidationError:
      properties:
        loc:
          items:
            anyOf:
              - type: string
              - type: integer
          type: array
          title: Location
        msg:
          type: string
          title: Message
        type:
          type: string
          title: Error Type
      type: object
      required:
        - loc
        - msg
        - type
      title: ValidationError


"""

---
# Итоговый вывод

1. Code-first - удобен для прототипирования MVP или создания инструментов внутреннего пользователя, но последовательная реализация может  затруднить создание робастной архитектуры (особенно если проект масштабен), приводя как к неоптимальным взаимодействиям узлов так и дублированию

2. Api-first - требует  знаний и понимания методов правильного редактирования сырой структуры API, что потенциально замедляет работу, но позволяет конструировать более проработанную и структурированную архитектуру, с оптимальными каналами взаимодействия.

---
## Итоговое оформление


1. Подготовьте ноутбук в логичной структуре: написание кода → работа с JSON → обработка ошибок → API в YAML → итоги.  
2. В ячейках Markdown сформулируйте 5–8 предложений с выводами, когда стоит применять подход Code-first и почему стоит придерживаться подхода API-first.  

